In [6]:
from typing import List, Tuple, Union, Set, Dict

In [7]:
from enum import Enum

class Operation(Enum):
    ADD = "+"
    SUBTRACT = "-"
    MULTIPLY = "*"
    DIVIDE = "/"
    POWER = "^"
    CONCAT = "><"

    __str__ = lambda self: self.value

CUMMUTATIVE = {Operation.ADD, Operation.MULTIPLY}

In [99]:
def phrase_order(phrase: List[Union[str, Operation]]) -> int:
    if isinstance(phrase, int):
        return phrase
    return min(
        value if isinstance(value, int) else float("inf") for value in phrase
    )

def canonicalize_phrase(phrase: List[Union[str, Operation]]) -> List[Union[str, Operation]]:
    if isinstance(phrase, int) or len(phrase) == 1:
        return phrase

    operation = phrase[0]
    left, right = phrase[1], phrase[2]
    try:
        left_operation = left[0]
    except TypeError:
        left_operation = None

    try:
        right_operation = right[0]
    except TypeError:
        right_operation = None

    if operation == Operation.MULTIPLY:
        if all([left_operation == Operation.DIVIDE, right_operation == Operation.DIVIDE]):
            pass
        elif left_operation == Operation.DIVIDE:
            numerator, denominator = left[1], left[2]
            phrase = tuple([Operation.DIVIDE, tuple([Operation.MULTIPLY, numerator, right]), denominator])
        elif right_operation == Operation.DIVIDE:
            numerator, denominator = right[1], right[2]
            phrase = tuple([Operation.DIVIDE, tuple([Operation.MULTIPLY, left, numerator]), denominator])

        if all([left_operation == Operation.MULTIPLY, right_operation == Operation.MULTIPLY]):
            subphrases = [left[1], left[2], right[1], right[2]]
            subphrases = sorted(subphrases, key=phrase_order)
            phrase = tuple([
                Operation.MULTIPLY,
                subphrases[0],
                tuple([
                    Operation.MULTIPLY,
                    subphrases[1],
                    tuple([
                        Operation.MULTIPLY,
                        subphrases[2],
                        subphrases[3]
                    ])
                ])
            ])
        elif left_operation == Operation.MULTIPLY:
            subphrases = [left[1], left[2], right]
            subphrases = sorted(subphrases, key=phrase_order)
            phrase = tuple([
                Operation.MULTIPLY,
                subphrases[0],
                tuple([
                    Operation.MULTIPLY,
                    subphrases[1],
                    subphrases[2]
                ])
            ])
        elif right_operation == Operation.MULTIPLY:
            subphrases = [left, right[1], right[2]]
            subphrases = sorted(subphrases, key=phrase_order)
            phrase = tuple([
                Operation.MULTIPLY,
                subphrases[0],
                tuple([
                    Operation.MULTIPLY,
                    subphrases[1],
                    subphrases[2]
                ])
            ])

    elif operation == Operation.ADD:
        if all([left_operation == Operation.SUBTRACT, right_operation == Operation.SUBTRACT]):
            pass
        elif left_operation == Operation.SUBTRACT:
            s_right, s_left = left[1], left[2]
            phrase = tuple([Operation.SUBTRACT, tuple([Operation.ADD, s_right, right]), s_left])
        elif right_operation == Operation.SUBTRACT:
            s_right, s_left = right[1], right[2]
            phrase = tuple([Operation.SUBTRACT, tuple([Operation.ADD, left, s_right]), s_left])

        if all([left_operation == Operation.ADD, right_operation == Operation.ADD]):
            subphrases = [left[1], left[2], right[1], right[2]]
            subphrases = sorted(subphrases, key=phrase_order)
            phrase = tuple([
                Operation.ADD,
                subphrases[0],
                tuple([
                    Operation.ADD,
                    subphrases[1],
                    tuple([
                        Operation.ADD,
                        subphrases[2],
                        subphrases[3]
                    ])
                ])
            ])
        elif left_operation == Operation.ADD:
            subphrases = [left[1], left[2], right]
            subphrases = sorted(subphrases, key=phrase_order)
            phrase = tuple([
                Operation.ADD,
                subphrases[0],
                tuple([
                    Operation.ADD,
                    subphrases[1],
                    subphrases[2]
                ])
            ])
        elif right_operation == Operation.ADD:
            subphrases = [left, right[1], right[2]]
            subphrases = sorted(subphrases, key=phrase_order)
            phrase = tuple([
                Operation.ADD,
                subphrases[0],
                tuple([
                    Operation.ADD,
                    subphrases[1],
                    subphrases[2]
                ])
            ])

    operation, left, right = phrase[0], phrase[1], phrase[2]
    left, right = canonicalize_phrase(left), canonicalize_phrase(right)
    phrase = tuple([operation, left, right])
    
    if operation in CUMMUTATIVE and phrase_order(phrase[1]) > phrase_order(phrase[2]):
        phrase = tuple([operation, phrase[2], phrase[1]])
            
    return phrase

def generate_phrase_combinations(
    fingerprint_dictionary_one: Dict[Tuple[int], Set[Tuple[Union[str, Operation]]]],
    fingerprint_dictionary_two: Dict[Tuple[int], Set[Tuple[Union[str, Operation]]]],
    operations: List[Operation]
) -> Dict[Tuple[int], Set[Tuple[Union[str, Operation]]]]:
    resulting_dictionary = {}

    for fingerprint_one, phrases_one in fingerprint_dictionary_one.items():
        for fingerprint_two, phrases_two in fingerprint_dictionary_two.items():
            fingerprint_one, fingerprint_two = set(fingerprint_one), set(fingerprint_two)
            if fingerprint_one & fingerprint_two:
                continue

            new_fingerprint = fingerprint_one | fingerprint_two
            new_fingerprint = tuple(sorted(new_fingerprint))
            new_phrases = set()

            for phrase_one in phrases_one:
                for phrase_two in phrases_two:
                    for operation in operations:
                        valid_phrases = [
                            tuple([operation, phrase_one, phrase_two]),
                            tuple([operation, phrase_two, phrase_one])
                        ]

                        for valid_phrase in valid_phrases:
                            valid_phrase = canonicalize_phrase(valid_phrase)
                            new_phrases.add(valid_phrase)

            if new_fingerprint in resulting_dictionary:
                resulting_dictionary[new_fingerprint] |= new_phrases
            else:
                resulting_dictionary[new_fingerprint] = new_phrases

    return resulting_dictionary

def generate_all_phrases(num_numbers: int, operations: List[str]) -> List[List[Union[int, Operation]]]:
    phrase_bank = {
        1: {
            tuple([i]): set([i]) for i in range(num_numbers)
        },
    }

    for current_phrase_length in range(1, num_numbers):
        for other_phrase_length in range(1, current_phrase_length + 1):
            if current_phrase_length + other_phrase_length > num_numbers:
                continue

            phrase_combinations = generate_phrase_combinations(
                phrase_bank[current_phrase_length],
                phrase_bank[other_phrase_length],
                operations
            )

            length = current_phrase_length + other_phrase_length
            if length in phrase_bank:
                for fingerprint, phrases in phrase_combinations.items():
                    if fingerprint in phrase_bank[length]:
                        phrase_bank[length][fingerprint] |= phrases
                    else:
                        phrase_bank[length][fingerprint] = phrases
            else:
                phrase_bank[length] = phrase_combinations

    return list(phrase_bank[num_numbers].values())[0]

In [9]:
def evaluate_permutation(
    permutation: List[Union[int, Operation]],
    numbers: List[int]
) -> int:
    if isinstance(permutation, int):
        return numbers[permutation]
    
    operation, left, right = permutation[0], permutation[1], permutation[2]
    left, right = evaluate_permutation(left, numbers), evaluate_permutation(right, numbers)
    if any([left is None, right is None]):
        return None

    if operation == Operation.ADD:
        result = left + right
    elif operation == Operation.SUBTRACT:
        result = left - right
    elif operation == Operation.MULTIPLY:
        result = left * right
    elif operation == Operation.DIVIDE:
        if right == 0:
            result = None
        else:
            result = left / right
            if not result.is_integer():
                result = None
            else:
                result = int(result)
    elif operation == Operation.POWER:
        if left <= 0 and right < 0:
            result = None
        else:
            result = left ** right
    elif operation == Operation.CONCAT:
        shift = 10 ** len(str(right))
        result = (left * shift) + right
    
    if not result is None and result > 100_000:
        return None
    
    return result
    

In [95]:
def pretty_print_expression(phrase: List[Union[int, Operation]], numbers: List[int]) -> str:
    if isinstance(phrase, int):
        return str(numbers[phrase])
    
    operation, left, right = phrase[0], phrase[1], phrase[2]
    left, right = pretty_print_expression(left, numbers), pretty_print_expression(right, numbers)
    
    if operation == Operation.ADD:
        return f"({left} + {right})"
    elif operation == Operation.SUBTRACT:
        return f"({left} - {right})"
    elif operation == Operation.MULTIPLY:
        return f"({left} * {right})"
    elif operation == Operation.DIVIDE:
        return f"({left} / {right})"
    elif operation == Operation.POWER:
        return f"({left} ^ {right})"
    elif operation == Operation.CONCAT:
        return f"({left} >< {right})"

In [96]:
def evaluate_all_permutations(
    phrases: List[Union[int, Operation]],
    numbers: List[int]
) -> Set[int]:
    results = []
    expressions = []
    for permutation in phrases:
        result = evaluate_permutation(permutation, numbers)
        if result is not None:
            results.append(result)
            expressions.append(pretty_print_expression(permutation, numbers))
    return results, expressions

In [107]:
phrases = generate_all_phrases(
    4,
    [
        Operation.ADD,
        Operation.SUBTRACT,
        Operation.MULTIPLY,
        Operation.DIVIDE,
        Operation.POWER,
    ]
)
len(phrases)

6526

In [110]:
numbers = [3, 3, 3, 3]
target = 24

numbers.sort()
results, expressions = evaluate_all_permutations(phrases, numbers)
num_equal_target = sum([result == target for result in results])
print(f"{num_equal_target} / {len(results)}")
print(num_equal_target / len(results))

for result, expression in zip(results, expressions):
    if result == target:
        print(expression)

4 / 4576
0.0008741258741258741
((3 * (3 * 3)) - 3)
((3 * (3 * 3)) - 3)
((3 * (3 * 3)) - 3)
((3 * (3 * 3)) - 3)


In [104]:
all_groups_of_four_digits = []
for a in range(9):
    for b in range(9):
        for c in range(9):
            for d in range(9):
                digits = [a + 1, b + 1, c + 1, d + 1]
                digits.sort()
                digits = tuple(digits)
                all_groups_of_four_digits.append(digits)

all_groups_of_four_digits = list(set(all_groups_of_four_digits))
print(len(all_groups_of_four_digits))

495


In [106]:
sum_equal_target = 0
total_possible = 0
for digits in all_groups_of_four_digits:
    results, _ = evaluate_all_permutations(phrases, digits)
    num_equal_target = sum([result == target for result in results])
    sum_equal_target += num_equal_target
    if num_equal_target > 0:
        total_possible += 1

print(sum_equal_target / total_possible)
print(total_possible / len(all_groups_of_four_digits))

8.317380352644836
0.802020202020202
